In [30]:
import pandas as pd
import os
import kenpompy
import plotly.graph_objects as go
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Returns an authenticated browser that can then be used to scrape pages that require authorization.
# Credentials are loaded from environment variables (stored in .env file)



True

In [31]:
import requests
import json

# API Configuration
# API key is loaded from environment variables (stored in .env file)
BASE_URL = "https://kenpom.com/api.php"
API_KEY = os.getenv('KENPOM_API_KEY')

headers = {
    "Authorization": f"Bearer {API_KEY}"
}

print("=" * 70)
print("KenPom API Demo - How to Use")
print("=" * 70)

# Example 1: Get Current Ratings for 2025 Season (Top 10 Teams)
print("\n1. CURRENT RATINGS - Top 10 Teams (2025 Season)")
print("-" * 70)
response = requests.get(
    f"{BASE_URL}?endpoint=ratings&y=2025",
    headers=headers
)
data = response.json()
df = pd.DataFrame(data)



KenPom API Demo - How to Use

1. CURRENT RATINGS - Top 10 Teams (2025 Season)
----------------------------------------------------------------------


In [38]:
import numpy as np
from matplotlib.path import Path as MPLPath

# Define trapezoid points: (64.5,20), (70.2,20), (62.5,40), (72,40)
trapezoid_points = np.array([
    [64.5, 20],  # bottom left
    [70.2, 20],  # bottom right
    [72, 40],    # top right
    [62.5, 40]   # top left
])

# Function to check if a point is inside the trapezoid
def point_in_polygon(point, polygon):
    path = MPLPath(polygon)
    return path.contains_point(point)

# Separate teams inside and outside the trapezoid
inside_trapezoid = []
outside_trapezoid = []
inside_data = []
outside_data = []

for idx, row in df.iterrows():
    point = (row['AdjTempo'], row['AdjEM'])
    if point_in_polygon(point, trapezoid_points):
        inside_trapezoid.append(idx)
        inside_data.append({
            'x': row['AdjTempo'],
            'y': row['AdjEM'],
            'name': row['TeamName']
        })
    else:
        outside_trapezoid.append(idx)
        outside_data.append({
            'x': row['AdjTempo'],
            'y': row['AdjEM'],
            'name': row['TeamName']
        })

# Create the plotly figure
fig = go.Figure()

# Add trapezoid shape
trapezoid_x = [64.5, 70.2, 72, 62.5, 64.5]  # Close the polygon
trapezoid_y = [20, 20, 40, 40, 20]

fig.add_trace(go.Scatter(
    x=trapezoid_x,
    y=trapezoid_y,
    fill='toself',
    fillcolor='rgba(0,0,0,0.1)',
    line=dict(color='black', width=3),
    mode='lines',
    name='Highlighted Zone (Trapezoid)',
    showlegend=True,
    hoverinfo='skip'
))

# Add points outside the trapezoid (blue dots)
if outside_data:
    fig.add_trace(go.Scatter(
        x=[d['x'] for d in outside_data],
        y=[d['y'] for d in outside_data],
        mode='markers',
        marker=dict(
            size=6,
            color='blue',
            symbol='circle',
            opacity=0.6,
            line=dict(width=0.5, color='darkblue')
        ),
        text=[d['name'] for d in outside_data],
        hovertemplate='<b>%{text}</b><br>' +
                      'Tempo: %{x:.1f}<br>' +
                      'AdjEM: %{y:.1f}<extra></extra>',
        name='Outside Trapezoid',
        showlegend=True
    ))

# Add points inside the trapezoid (yellow stars)
if inside_data:
    fig.add_trace(go.Scatter(
        x=[d['x'] for d in inside_data],
        y=[d['y'] for d in inside_data],
        mode='markers',
        marker=dict(
            size=10,
            color='gold',
            symbol='star',
            opacity=0.9,
            line=dict(width=1, color='orange')
        ),
        text=[d['name'] for d in inside_data],
        hovertemplate='<b>%{text}</b><br>' +
                      'Tempo: %{x:.1f}<br>' +
                      'AdjEM: %{y:.1f}<extra></extra>',
        name='Inside Trapezoid',
        showlegend=True
    ))

# Update layout with white theme
fig.update_layout(
    title=dict(
        text='NCAA March Madness: KenPom Team Ratings<br>Tempo vs Efficiency Margin',
        font=dict(size=20, color='black', family='Arial Black'),
        x=0.5
    ),
    xaxis=dict(
        title=dict(
            text='Adjusted Tempo',
            font=dict(size=16, color='black', family='Arial Black')
        ),
        tickfont=dict(size=12, color='black'),
        gridcolor='rgba(0,0,0,0.1)',
        gridwidth=1,
        showgrid=True,
        zeroline=False
    ),
    yaxis=dict(
        title=dict(
            text='Adjusted Efficiency Margin',
            font=dict(size=16, color='black', family='Arial Black')
        ),
        tickfont=dict(size=12, color='black'),
        gridcolor='rgba(0,0,0,0.1)',
        gridwidth=1,
        showgrid=True,
        zeroline=False
    ),
    plot_bgcolor='white',
    paper_bgcolor='white',
    width=1200,
    height=800,
    legend=dict(
        x=1.02,
        y=1,
        bgcolor='white',
        bordercolor='black',
        borderwidth=2,
        font=dict(size=12, color='black', family='Arial')
    ),
    hovermode='closest'
)

fig.show()

print(f"\nTeams inside the trapezoid: {len(inside_trapezoid)}")
if inside_trapezoid:
    print("\nHighlighted Teams:")
    for idx in inside_trapezoid:
        print(f"  - {df.loc[idx, 'TeamName']} (Tempo: {df.loc[idx, 'AdjTempo']:.1f}, AdjEM: {df.loc[idx, 'AdjEM']:.1f})")


Teams inside the trapezoid: 26

Highlighted Teams:
  - Duke (Tempo: 66.0, AdjEM: 39.3)
  - Florida (Tempo: 70.1, AdjEM: 36.5)
  - Auburn (Tempo: 68.2, AdjEM: 35.1)
  - Tennessee (Tempo: 63.7, AdjEM: 30.9)
  - Michigan St. (Tempo: 67.5, AdjEM: 28.5)
  - Gonzaga (Tempo: 70.5, AdjEM: 28.0)
  - Texas Tech (Tempo: 66.1, AdjEM: 27.9)
  - Maryland (Tempo: 69.6, AdjEM: 27.1)
  - Iowa St. (Tempo: 68.8, AdjEM: 26.7)
  - Wisconsin (Tempo: 68.0, AdjEM: 26.6)
  - Arizona (Tempo: 70.3, AdjEM: 26.4)
  - St. John's (Tempo: 70.0, AdjEM: 26.1)
  - Purdue (Tempo: 65.1, AdjEM: 25.4)
  - Texas A&M (Tempo: 66.8, AdjEM: 23.7)
  - Missouri (Tempo: 68.6, AdjEM: 23.6)
  - UCLA (Tempo: 65.0, AdjEM: 23.5)
  - Michigan (Tempo: 70.1, AdjEM: 23.4)
  - Mississippi (Tempo: 68.1, AdjEM: 23.3)
  - Clemson (Tempo: 64.6, AdjEM: 23.2)
  - Kansas (Tempo: 68.6, AdjEM: 22.9)
  - BYU (Tempo: 67.6, AdjEM: 22.3)
  - Oregon (Tempo: 68.1, AdjEM: 21.8)
  - Louisville (Tempo: 69.1, AdjEM: 21.7)
  - Marquette (Tempo: 67.7, AdjEM: 21